In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch import nn,optim
from torch.nn import functional as F 
from torch.utils.data import Dataset,DataLoader,TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [101]:
traindata = pd.read_csv("newresult.csv")
trainlabel = pd.read_csv("newlabel.csv")
traindata = traindata.drop(['Unnamed: 0'],axis = 1).values
trainlabel = trainlabel['func'].values

In [102]:
X_train, X_test, Y_train, Y_test = train_test_split(traindata, trainlabel, test_size = 0.2, random_state = 1)
X_train, Y_train = torch.FloatTensor(X_train), torch.FloatTensor(Y_train)
X_test, Y_test = torch.FloatTensor(X_test), torch.FloatTensor(Y_test)

In [103]:
Y_train

tensor([340.6172,   3.7677,  -0.8499,  ...,   0.9953,   2.8557,   6.4792])

In [104]:
batchsize = 32
train_dataset = torch.utils.data.TensorDataset(X_train, Y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, Y_test)

#dataloader 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batchsize, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batchsize, shuffle = False)

In [105]:
class CNN_16d(nn.Module):
    def __init__(self):
        super(CNN_16d,self).__init__()
        self.conv1 = nn.Conv1d(1, 2, 2, padding=0, stride=2)
        #self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv1d(2, 4, 2, stride=2)
        self.conv3 = nn.Conv1d(4, 6, 2, stride=2)
        self.conv4 = nn.Conv1d(6, 3, 2, stride=2)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(3, 60)
        self.fc2 = nn.Linear(60, 20)
        self.fc3 = nn.Linear(20, 1)
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)  
        x = self.flat(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [106]:
model = CNN_16d()
if torch.cuda.is_available():
    print("gpu_available")
    model.cuda()

gpu_available


In [107]:
print(model)

CNN_16d(
  (conv1): Conv1d(1, 2, kernel_size=(2,), stride=(2,))
  (conv2): Conv1d(2, 4, kernel_size=(2,), stride=(2,))
  (conv3): Conv1d(4, 6, kernel_size=(2,), stride=(2,))
  (conv4): Conv1d(6, 3, kernel_size=(2,), stride=(2,))
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=3, out_features=60, bias=True)
  (fc2): Linear(in_features=60, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=1, bias=True)
)


In [108]:
X = torch.randn(1,1,16)
xiao = CNN_16d()
xiao(X)

tensor([[-0.0301]], grad_fn=<AddmmBackward0>)

In [110]:
optimizer = optim.Adam(model.parameters(), lr =0.001)
criteon= nn.MSELoss()
EPOCH = 20
loss_list = []
for epoch in range(EPOCH):
    model.train()
    sum_loss = 0 
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        #data, target = Variable(data), Variable(target)

        data = torch.reshape(data,(batchsize, 1, 16))
        target = torch.reshape(target, (batchsize, 1))
        optimizer.zero_grad()
        
        
        data=data.to(torch.float32)
        target=target.to(torch.float32)
        output = model(data)

        
        #print(data)
        #print(target)
        #print("The output is:")
        #print(output)
        
        loss = criteon(output, target)
        loss.backward()
        optimizer.step()

        sum_loss += loss.item()
        if batch_idx % 10 == 0:
            print("train epoch {} loss : {:.3f}".format(epoch, sum_loss/100))
            loss_list.append(sum_loss/100)
            sum_loss = 0         

train epoch 0 loss : 449.202
train epoch 0 loss : 4030.904
train epoch 0 loss : 4740.638
train epoch 0 loss : 5511.438
train epoch 0 loss : 4265.532
train epoch 0 loss : 3860.532
train epoch 0 loss : 4744.626
train epoch 0 loss : 4006.230
train epoch 0 loss : 3738.941
train epoch 0 loss : 3169.693
train epoch 0 loss : 3638.912
train epoch 0 loss : 3137.901
train epoch 0 loss : 2858.113
train epoch 0 loss : 2454.673
train epoch 0 loss : 2932.392
train epoch 0 loss : 2597.797
train epoch 0 loss : 2447.735
train epoch 0 loss : 2161.510
train epoch 0 loss : 1815.357
train epoch 0 loss : 1893.842
train epoch 0 loss : 2107.532
train epoch 0 loss : 1611.028
train epoch 0 loss : 1589.533
train epoch 0 loss : 1881.998
train epoch 0 loss : 1554.393
train epoch 1 loss : 134.484
train epoch 1 loss : 1641.655
train epoch 1 loss : 1664.962
train epoch 1 loss : 1458.882
train epoch 1 loss : 1604.735
train epoch 1 loss : 1642.154
train epoch 1 loss : 1312.273
train epoch 1 loss : 1235.233
train epoch 

In [111]:
model = model.cpu()
model(torch.Tensor(X_test[0].reshape(1,1,16))).item()

364.27740478515625

In [112]:
#test数据集上的表现
testing = pd.DataFrame(X_test)
#testing['truth'] = Y_test


In [113]:
re = model(torch.Tensor(X_test.reshape(2000,1,16))).detach().numpy()
re = re.reshape(2000)
true = Y_test.numpy()
dataframe = pd.DataFrame([re, true]).T
dataframe.columns = ['pred','truth']
dataframe['error'] = np.abs(dataframe['pred'] - dataframe['truth'])
dataframe['percentage'] = dataframe['error']/np.abs(dataframe['truth'] )

In [118]:
dataframe['percentage'].mean()

1.141919788293053

In [119]:
dataframe

,pred,truth,error,percentage
0,364.277435,362.790985,1.486450,0.004097
1,2.721594,0.204353,2.517241,12.318089
2,2.880503,0.775139,2.105364,2.716111
3,446.227753,448.795654,2.567902,0.005722
4,301.720673,299.815735,1.904938,0.006354
...,...,...,...,...
1995,409.090240,407.261139,1.829102,0.004491
1996,106.116310,107.372185,1.255875,0.011696
1997,15.442434,15.047541,0.394894,0.026243
1998,242.997787,239.743607,3.254181,0.013574


In [120]:
import seaborn as sns 
import matplotlib.pyplot as plt 